In [9]:
###################################################################################################
#
# PairIdentification.py
#
# Copyright (C) by Andreas Zoglauer & Harrison Costatino.
#
# Please see the file LICENSE in the main repository for the copyright-notice.
#
###################################################################################################



###################################################################################################

import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

import numpy as np

#from mpl_toolkits.mplot3d import Axes3D
#import matplotlib.pyplot as plt

import random

import signal
import sys
import time
import math
import csv
import os
import argparse
from datetime import datetime
from functools import reduce


print("\nPair Identification")
print("============================\n")



# Step 1: Input parameters
###################################################################################################


# Default parameters

UseToyModel = True

# Split between training and testing data
TestingTrainingSplit = 0.1

MaxEvents = 10

# File names
FileName = "PairIdentification.p1.sim.gz"
GeometryName = "$(MEGALIB)/resource/examples/geomega/GRIPS/GRIPS.geo.setup"


# Set in stone later
TestingTrainingSplit = 0.8

OutputDirectory = "Results"


parser = argparse.ArgumentParser(description='Perform training and/or testing of the pair identification machine learning tools.')
parser.add_argument('-f', '--filename', default='PairIdentification.p1.sim.gz', help='File name used for training/testing')
parser.add_argument('-m', '--maxevents', default='1000', help='Maximum number of events to use')
parser.add_argument('-s', '--testingtrainigsplit', default='0.1', help='Testing-training split')
parser.add_argument('-b', '--batchsize', default='128', help='Batch size')

args = parser.parse_args()

if args.filename != "":
  FileName = args.filename

if int(args.maxevents) > 1000:
  MaxEvents = int(args.maxevents)

if int(args.batchsize) >= 16:
  BatchSize = int(args.batchsize)

if float(args.testingtrainigsplit) >= 0.05:
  TestingTrainingSplit = float(args.testingtrainigsplit)


if os.path.exists(OutputDirectory):
  Now = datetime.now()
  OutputDirectory += Now.strftime("_%Y%m%d_%H%M%S")

os.makedirs(OutputDirectory)



###################################################################################################
# Step 2: Global functions
###################################################################################################


# Take care of Ctrl-C
Interrupted = False
NInterrupts = 0
def signal_handler(signal, frame):
  global Interrupted
  Interrupted = True
  global NInterrupts
  NInterrupts += 1
  if NInterrupts >= 2:
    print("Aborting!")
    sys.exit(0)
  print("You pressed Ctrl+C - waiting for graceful abort, or press  Ctrl-C again, for quick exit.")
signal.signal(signal.SIGINT, signal_handler)


# Everything ROOT related can only be loaded here otherwise it interferes with the argparse
from EventData import EventData

# Load MEGAlib into ROOT so that it is usable
import ROOT as M
M.gSystem.Load("$(MEGALIB)/lib/libMEGAlib.so")
M.PyConfig.IgnoreCommandLineOptions = True



###################################################################################################
# Step 3: Create some training, test & verification data sets
###################################################################################################


# Read the simulation file data:
DataSets = []
NumberOfDataSets = 0

if UseToyModel == True:
  for e in range(0, MaxEvents):
    Data = EventData()
    Data.createFromToyModel(e)
    DataSets.append(Data)
    
    NumberOfDataSets += 1
    if NumberOfDataSets > 0 and NumberOfDataSets % 1000 == 0:
      print("Data sets processed: {}".format(NumberOfDataSets))
  
else:
  # Load geometry:
  Geometry = M.MDGeometryQuest()
  if Geometry.ScanSetupFile(M.MString(GeometryName)) == True:
    print("Geometry " + GeometryName + " loaded!")
  else:
    print("Unable to load geometry " + GeometryName + " - Aborting!")
    quit()


  Reader = M.MFileEventsSim(Geometry)
  if Reader.Open(M.MString(FileName)) == False:
    print("Unable to open file " + FileName + ". Aborting!")
    quit()


  print("\n\nStarted reading data sets")
  NumberOfDataSets = 0
  while NumberOfDataSets < MaxEvents:
    Event = Reader.GetNextEvent()
    if not Event:
      break

    if Event.GetNIAs() > 0:
      Data = EventData()
      if Data.parse(Event) == True:
        if Data.hasHitsOutside(XMin, XMax, YMin, YMax, ZMin, ZMax) == False:
          DataSets.append(Data)
          NumberOfDataSets += 1
          if NumberOfDataSets % 500 == 0:
            print("Data sets processed: {}".format(NumberOfDataSets))

print("Info: Parsed {} events".format(NumberOfDataSets))

# Split the data sets in training and testing data sets

TestingTrainingSplit = 0.75


numEvents = len(DataSets)

numTraining = int(numEvents * TestingTrainingSplit)

TrainingDataSets = DataSets[:numTraining]
TestingDataSets = DataSets[numTraining:]



# For testing/validation split
# ValidationDataSets = TestingDataSets[:int(len(TestingDataSets)/2)]
# TestingDataSets = TestingDataSets[int(len(TestingDataSets)/2):]

print("###### Data Split ########")
print("Training/Testing Split: {}".format(TestingTrainingSplit))
print("Total Data: {}, Training Data: {},Testing Data: {}".format(numEvents, len(TrainingDataSets), len(TestingDataSets)))
print("##########################")


###################################################################################################
# Step 4: Setting up the neural network
###################################################################################################



###################################################################################################
# Step 5: Training and evaluating the network
###################################################################################################



Pair Identification

Start: 3.853819189130032, 6.44699536183873, -7
Event ID: 0
  Origin Z: -7
  Gamma Energy: 9999.999999999998
  Hit 1 (origin: 0): type=m, pos=(3.853819189130032, 6.44699536183873, -7.0)cm, E=466.5794975292707keV
  Hit 2 (origin: 1): type=e, pos=(4.000801716031608, 6.445609290686664, -8.0)cm, E=373.80874572837524keV
  Hit 3 (origin: 2): type=e, pos=(4.092914210559528, 6.4434291032139095, -9.0)cm, E=433.22361034520776keV
  Hit 4 (origin: 3): type=e, pos=(4.24660640256076, 6.310608856452847, -10.0)cm, E=490.7422320329567keV
  Hit 5 (origin: 4): type=e, pos=(4.364601044121417, 6.524434490970692, -11.0)cm, E=522.5960532446603keV
  Hit 6 (origin: 5): type=e, pos=(4.4222339283507175, 6.902977951329251, -12.0)cm, E=578.3317764959277keV
  Hit 7 (origin: 6): type=e, pos=(4.602535591235732, 7.500212336309487, -13.0)cm, E=625.6969861735481keV
  Hit 8 (origin: 7): type=e, pos=(4.162024414131495, 8.118272337142454, -14.0)cm, E=655.7276803065596keV
  Hit 9 (origin: 8): type=e, po

In [26]:
def generate_incidence(edges, pos_data):
    #Generate Incidence Matrix from Edge List
    n_hits = len(pos_data)
    n_edges = len(edges)
    Ri = np.zeros((n_hits, n_edges), dtype=np.uint8)
    Ro = np.zeros((n_hits, n_edges), dtype=np.uint8)
    
    for i in range(len(edges)):
        point = edges[i]
        from_pt = point[0]
        to_pt = point[1]
        Ro[from_pt][i] = 1
        Ri[to_pt][i] = 1
    
    return Ri, Ro

In [27]:
def connect_pos(pos_data):
    #Manually Connect Graph based on Positions
    edges = []
    for i in range(len(pos_data)):
        point_A = pos_data[i]
        z_A = point_A[2]

        for j in range(len(pos_data)):
            point_B = pos_data[j]
            z_B = point_B[2]

            if z_B == z_A + 1:
                edges.append((i, j))
                edges.append((j, i))
    return generate_incidence(edges, pos_data)

In [28]:
def pad(arr, shape):
    #Padd arr to Shape
    padded_arr = np.zeros(shape)
    if len(shape) == 1:
        padded_arr[:arr.shape[0]] = arr
    elif len(shape) == 2:
        padded_arr[:arr.shape[0],:arr.shape[1]] = arr
    return padded_arr

In [104]:
def vectorize_data(eventArr):
    # Edge Validity Labels, Manually Connected Rin, Mannually Connected Rout, XYZ, Type, Energy, Gamma Energy
    Edge_Labels, True_Ri, True_Ro, Man_Ri, Man_Ro, XYZ, Type, Energy, GammaEnergy = [], [], [], [], [], [], [], [], []
    max_hits, max_edges = 0, 0
    
    #Start Parsing Events
    for event in eventArr:
        #Keep track of max hits for padding
        max_hits = max(max_hits, len(event.X))
        
        #Generate Incidence Matrices based on Edges
        edges = []
        pos = np.swapaxes(np.vstack((event.X, event.Y, event.Z)), 0, 1)
        for i in range(1,len(event.Origin+1)):
            edges.append((i-1,event.Origin[i-1]-1))
        print(edges)
        e_Ri, e_Ro = generate_incidence(edges,pos)
        
        #Generate Proposed Incidence Matrices based on Positions
        p_Ri, p_Ro = connect_pos(pos)
        
        #Generate Edge Labels (0 - fake edge; 1 - true edge)
        e_label = np.zeros(p_Ri.shape[1])
        for i in range(p_Ri.shape[1]):
            out = np.where(p_Ro[:,i] == 1)
            print(out)
            inn = np.where(p_Ri[:,i] == 1)
            print(int)
            e_label[i] = 1*((out, inn) in edges)
            
        #Keep track of max edges for Padding
        max_edges = max(max_edges, p_Ri.shape[1])
        
        #Add all of the event data to lists
        Edge_Labels.append(e_label)
        True_Ri.append(e_Ri)
        True_Ro.append(e_Ro)
        Man_Ri.append(p_Ri)
        Man_Ro.append(p_Ro)
        XYZ.append(np.vstack((event.X, event.Y, event.Z)).T)
        Type.append(2*(event.Type=='m')+(event.Type=='p'))
        Energy.append(event.E)
        GammaEnergy.append(event.GammaEnergy)
    
    #Padding based on Max Hits and Max Edges
    for i in range(len(Ri)):
        Edge_Labels[i] = pad(Edge_Labels[i],(max_edges,))
        Man_Ri[i] = pad(Man_Ri[i],(max_hits,max_edges))
        Man_Ro[i] = pad(Man_Ro[i],(max_hits,max_edges))
        XYZ[i] = pad(XYZ[i],(max_hits,3))
        Type[i] = pad(Type[i],(max_hits,))
        Energy[i] = pad(Energy[i],(max_hits,))
    
    return np.array(Edge_Labels), np.array(Man_Ri), np.array(Man_Ro), np.array(XYZ), np.array(Type), np.array(Energy), np.array(GammaEnergy)

In [105]:
Edge_Labels, Man_Ri, Man_Ro, XYZ, Type, Energy, GammaEnergy = vectorize_data(TrainingDataSets)

[(0, -1), (1, 0), (2, 1), (3, 2), (4, 3), (5, 4), (6, 5), (7, 6), (8, 7), (9, 8), (10, 9), (11, 10), (12, 11), (13, 12), (14, 0)]
(array([1]),)
<class 'int'>
(array([0]),)
<class 'int'>
(array([1]),)
<class 'int'>
(array([15]),)
<class 'int'>
(array([2]),)
<class 'int'>
(array([1]),)
<class 'int'>
(array([2]),)
<class 'int'>
(array([14]),)
<class 'int'>
(array([3]),)
<class 'int'>
(array([2]),)
<class 'int'>
(array([4]),)
<class 'int'>
(array([3]),)
<class 'int'>
(array([5]),)
<class 'int'>
(array([4]),)
<class 'int'>
(array([6]),)
<class 'int'>
(array([5]),)
<class 'int'>
(array([7]),)
<class 'int'>
(array([6]),)
<class 'int'>
(array([8]),)
<class 'int'>
(array([7]),)
<class 'int'>
(array([9]),)
<class 'int'>
(array([8]),)
<class 'int'>
(array([9]),)
<class 'int'>
(array([10]),)
<class 'int'>
(array([9]),)
<class 'int'>
(array([12]),)
<class 'int'>
(array([10]),)
<class 'int'>
(array([7]),)
<class 'int'>
(array([11]),)
<class 'int'>
(array([8]),)
<class 'int'>
(array([11]),)
<class 'i

In [97]:
Features = [[XYZ[i],Man_Ri[i], Man_Ro[i]] for i in range(XYZ.shape[0])]
Labels = Edge_Labels

In [98]:
Man_Ri.shape

(7, 17, 62)

In [89]:
import torch.distributed as dist
from torch.utils.data import DataLoader
from torch.utils.data.distributed import DistributedSampler

# Locals
from datasets import get_data_loaders
from trainers import get_trainer

In [48]:
trainer = get_trainer('gnn')
trainer.build_model()
trainer.print_model_summary()

In [49]:
train_data_loader = DataLoader(train_dataset, batch_size=1)
test_data_loader = DataLoader(valid_dataset, batch_size=1)

NameError: name 'train_dataset' is not defined

In [ ]:
summary = trainer.train(train_data_loader=train_data_loader,
                        valid_data_loader=valid_data_loader,
                        **train_config)

In [99]:
TrainingDataSets[0].print()

Event ID: 0
  Origin Z: -7
  Gamma Energy: 9999.999999999998
  Hit 1 (origin: 0): type=m, pos=(3.853819189130032, 6.44699536183873, -7.0)cm, E=466.5794975292707keV
  Hit 2 (origin: 1): type=e, pos=(4.000801716031608, 6.445609290686664, -8.0)cm, E=373.80874572837524keV
  Hit 3 (origin: 2): type=e, pos=(4.092914210559528, 6.4434291032139095, -9.0)cm, E=433.22361034520776keV
  Hit 4 (origin: 3): type=e, pos=(4.24660640256076, 6.310608856452847, -10.0)cm, E=490.7422320329567keV
  Hit 5 (origin: 4): type=e, pos=(4.364601044121417, 6.524434490970692, -11.0)cm, E=522.5960532446603keV
  Hit 6 (origin: 5): type=e, pos=(4.4222339283507175, 6.902977951329251, -12.0)cm, E=578.3317764959277keV
  Hit 7 (origin: 6): type=e, pos=(4.602535591235732, 7.500212336309487, -13.0)cm, E=625.6969861735481keV
  Hit 8 (origin: 7): type=e, pos=(4.162024414131495, 8.118272337142454, -14.0)cm, E=655.7276803065596keV
  Hit 9 (origin: 8): type=e, pos=(4.825665453363485, 10.179460903846278, -15.0)cm, E=722.75377851815

In [100]:
XYZ[0]

array([[  3.85381919,   6.44699536,  -7.        ],
       [  4.00080172,   6.44560929,  -8.        ],
       [  4.09291421,   6.4434291 ,  -9.        ],
       [  4.2466064 ,   6.31060886, -10.        ],
       [  4.36460104,   6.52443449, -11.        ],
       [  4.42223393,   6.90297795, -12.        ],
       [  4.60253559,   7.50021234, -13.        ],
       [  4.16202441,   8.11827234, -14.        ],
       [  4.82566545,  10.1794609 , -15.        ],
       [  3.72365243,   8.82462364, -16.        ],
       [  8.95757892,  20.92838955, -15.        ],
       [  6.81184758,  18.17600153, -16.        ],
       [  9.25953755,  17.88431744, -15.        ],
       [  9.6855503 ,  18.82045685, -16.        ],
       [  1.52745594,   5.78874964,  -8.        ],
       [  1.91741115,   6.67391267,  -7.        ],
       [  0.        ,   0.        ,   0.        ]])

In [101]:
Man_Ri[0]

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [102]:
Man_Ro[0]

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [103]:
Edge_Labels[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [107]:
a = np.array([[1,2,6],[4,5,6]])

In [114]:
np.where(a[:,2] == 6)

(array([1]),)

In [115]:
a[:,2]

array([3, 6])